In [2]:
########################################################################################################################
## -- libraries and packages -- ########################################################################################
########################################################################################################################
import calm

########################################################################################################################
## -- create text files from csv and load the data as text files -- ####################################################
########################################################################################################################
data_handler_module = calm.data_handler.DataHandler()
path = "data/dataset/"

data_handler_module.convert_dailydialog_to_txt(path + "train.csv", path + "train.txt")
data_handler_module.convert_dailydialog_to_txt(path + "test.csv", path + "test.txt")
data_handler_module.convert_dailydialog_to_txt(path + "validation.csv", path + "validation.txt")

tr_data = data_handler_module.load_data(path + "train.txt")
te_data = data_handler_module.load_data(path + "test.txt")
va_data = data_handler_module.load_data(path + "validation.txt")

########################################################################################################################
## -- setting up the tokenizer vocab and most common pair lists -- #####################################################
########################################################################################################################

## -- run this if you are generating from scratch (takes around 10 - minutes) -- ##
# tokenizer = calm.tokenization.Tokenizer(tr_data)
# tokenizer.max_vocab_size = 1024
# tokenizer.save_state_path = "./data/vocab/tokenizer_state.pkl"
# tokenizer.generate_vocab()

## -- run this if you are loading what you have already generated (recommended after the first run) -- ##
tokenizer = calm.tokenization.Tokenizer()
tokenizer.load_state("./data/vocab/tokenizer_state.pkl")

In [4]:
txt = te_data[:1000]
txt_new = tokenizer.encoder(txt)

print(len(txt), len(txt_new))

1000 270
